In [1]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
sns.set()

## Filter out non automotive-related alliances

In [3]:
df = pd.read_csv('../data.csv')
df = df[df['psic'].notna()] #drop 45 NaN participant SIC, fix the issue by not solving it
autoDF = df[df['psicp'].str.contains('3711')] #Create df who have at least one participant in the automotive industry with SIC: 3711
autoDF = autoDF.drop('Unnamed: 0', axis=1)
autoDF = autoDF.reset_index(drop=True)

#Export
autoDF.to_csv('./data/autodf.csv', index=False) #create csv sample

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_36208/2145547248.py:1: DtypeWarning: Columns (3,4,7,8,11,12,14,15,16,19,45,46,49,56,69,83,95,98,99,100,104,108,109,119,120,121,122,123,124,125,127,128,129,141,143,144,145,146,147,148,155,188,217,218,219,220,221,222,223,224,226,229,232,233,234,235,236,237,238,239,240,243,244,245,246,248,249,250,252) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data.csv')


## Data selection

### Filter variables

In [4]:
variables = ["id","activity", "activityc", "da", "jvinc", "jvindustry", "jvstatus", "p", "pbl", 
		   "pbuss", "sicp", "sic", "SICPDESC".lower(), "psic", "psicp", "nump", "jvf", "jvtype", 
		   "rndf", "pemp", "PBUSSOURCE".lower(), "HITECHC".lower(), "crlic", "crtech", "TECHNIQUEC".lower(),
		   "TECHTR".lower(), "PPUBC".lower()]
filteredDF = autoDF[variables]

### Select timeframe

In [5]:
filteredDF['da'] = pd.to_datetime(filteredDF['da']) 
timedDF = filteredDF[(filteredDF['da'] >= '2002-01-01') & (filteredDF['da'] <= '2012-12-31')]
timedDF.set_index('da', inplace=True, drop=True)

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_36208/2738038061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filteredDF['da'] = pd.to_datetime(filteredDF['da'])


## Feature engineering

### Split PSICPs to seperate columns

In [6]:
pattern = r'\\n' #Regex pattern to split name

for n in range(1, (int(timedDF["nump"].max()) + 1)): 
	timedDF['p' + str(n) + "name"] =  timedDF['p'].str.split(pattern, expand=True)[n-1] #Create seperate columns for each possible participant

	timedDF["p" + str(n) + "sicp"] = timedDF['psicp'].str.split(pattern, expand=True)[n-1] #Create seperate placeholder columns to split participant SIC codes

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_36208/1130787018.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF['p' + str(n) + "name"] =  timedDF['p'].str.split(pattern, expand=True)[n-1] #Create seperate columns for each possible participant
/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_36208/1130787018.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF["p" + str(n) + "sicp"] = timedDF['psicp'].str.split(pattern, expand=True)[n-1] #Create seperate placeholder c

### Define incumbent / new-entrant alliance

In [7]:
cols = timedDF.filter(regex='p\d+sicp', axis=1).columns #Get columns of individual participant SICs

timedDF['pdynamic'] = (timedDF[cols]=='3711').sum(axis=1) > 1 #Flag True for alliances consisting of more than one automotive incumbent
timedDF['pdynamic'] = timedDF['pdynamic'].map({True: "incumbent", False: "new_entrant"}) #Turn tha above from boolean to string

/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_36208/3940381924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF['pdynamic'] = (timedDF[cols]=='3711').sum(axis=1) > 1 #Flag True for alliances consisting of more than one automotive incumbent
/var/folders/5h/tc1b2dhx483_cx6pj09_pyc80000gn/T/ipykernel_36208/3940381924.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timedDF['pdynamic'] = timedDF['pdynamic'].map({True: "incumbent", False: "new_entrant"}) #Turn tha above from boolean to

In [8]:
#Export
timedDF.to_csv('./data/JV_data_2002-2012.csv')

### Generate discontintuiy time distance variations

In [9]:
JV_data_dist = timedDF.copy() ##Make copy

disc_day = pd.to_datetime("02/06/2007", format="%d/%m/%Y")

#### Integer distance

In [10]:
JV_data_dist['ddist_int'] = ((disc_day - JV_data_dist.index).days) * -1 #time -1 to reverse days count, negative should be before 2007

#### Natural num distance (absolute)

In [11]:
JV_data_dist['ddist_abs'] = abs((disc_day - JV_data_dist.index).days)

#### Binary distance

In [12]:
#Let 0 describe alliances before discontinuity day
#Let 1 describe alliances after discontinuity day

JV_data_dist['ddist_bin'] = 0 #Create placeholder columnc
JV_data_dist.loc[JV_data_dist.index > disc_day, 'ddist_bin'] = 1

### Split participant employee count to seperate variables

In [51]:
pattern = r'\\n'
pattern2 = r'p\d+emp'

for n in range(1, (int(JV_data_dist["nump"].max()) + 1)): 
	JV_data_dist["p" + str(n) + "emp"] =  pd.to_numeric(JV_data_dist['pemp'].str.split(pattern, expand=True)[n-1]) #Create seperate columns for each possible participant
	JV_data_dist["p" + str(n) + "emp"].replace(to_replace='', value=None, inplace=True) #Replace unknown employee count with na

foo = JV_data_dist.filter(regex=pattern2, axis=1).notna() #Get columns of individual participant employee count

JV_data_dist['known_emp'] = foo.any(axis=1) #Binary if employee count is known for >= participant
JV_data_dist['known_allemp'] = JV_data_dist['nump'] == foo.sum(axis=1) #Binary if employee count is known for ALL participants

#Get log of AVG employess per participant
JV_data_dist['avg_emp_pp'] = np.log(JV_data_dist.filter(regex=pattern2, axis=1).mean(axis=1, skipna=True))

### Count public companies per alliance

In [60]:
pattern = r'\\n'

JV_data_dist['puplic_count'] = (JV_data_dist['ppubc'].str.split(pattern, expand=True) == 'P').sum(axis=1)

In [61]:
#Export
JV_data_dist.to_csv("./data/JV_data_dist.csv")

In [62]:
pd.options.display.max_columns = None
JV_data_dist

,id,activity,activityc,jvinc,jvindustry,jvstatus,p,pbl,pbuss,sicp,sic,sicpdesc,psic,psicp,nump,jvf,jvtype,rndf,pemp,pbussource,hitechc,crlic,crtech,techniquec,techtr,ppubc,p1name,p1sicp,p2name,p2sicp,p3name,p3sicp,p4name,p4sicp,p5name,p5sicp,p6name,p6sicp,p7name,p7sicp,p8name,p8sicp,p9name,p9sicp,pdynamic,ddist_int,ddist_abs,ddist_bin,p1emp,p2emp,p3emp,p4emp,p5emp,p6emp,p7emp,p8emp,p9emp,known_emp,known_allemp,avg_emp_pp,puplic_count
da,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2002-01-01,88623.0,Financial Services,FIN,DC,Credit Institutions,Completed/Signed,Toyota Motor Corp\nMetrobank,"Toyota Motor Corp,\nheadquartered in Aichi, Ja...",Mnfr passenger motor vehicles\nBank,6141,6141\n6153\n6159,NaN,3711\n3714\n6000,3711\n6000,2.0,Yes,NF,No,69478\n12114,Asian Wall Street Journal\nDuns Asia/Pacific-K...,0,N,N,NaN,N,P\nP,Toyota Motor Corp,3711,Metrobank,6000,None,None,None,None,None,None,None,None,None,None,None,None,None,None,new_entrant,-1978,1978,0,69478.0,12114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,10.616339,2
2002-01-03,88649.0,Manufacturing Services,MNF,AN,Machinery,Completed/Signed,SAIC\nYuejin Automobile Group Co\nTeksid SpA,"Shanghai Automotive Industry\nCorp (Group), lo...","Mnfr,whl motor vehicles,parts\nMnfr motor vehi...",3561,3561\n3593\n3491\n3714\n2796,NaN,3711\n3537\n3714\n6141\n6153\n6159\n7538\n5012...,3711\n3711\n3312,3.0,Yes,NF,No,\n\n,"Reuters\nEstimation\nDun's 10,000 Italian Cos",0,N,N,NaN,N,V\nV\nS,SAIC,3711,Yuejin Automobile Group Co,3711,Teksid SpA,3312,None,None,None,None,None,None,None,None,None,None,None,None,incumbent,-1976,1976,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,0
2002-01-04,88660.0,Property Development Services\nConstruction Se...,PDS\nCSN\nREI,DD,Real Estate; Mortgage Bankers and Brokers,Letter of Intent,Raba Automotive Holding Plc\nEngel General Dev...,"Raba Automotive Holding Plc,\nlocated in Gyor,...","Mnfr,whl commercial vehicles\nRE dvlp firm",6552,6552\n6512,NaN,3711\n3713\n3465\n3519\n3714\n3621\n3511\n5012...,3711\n6552,2.0,Yes,NF,No,\n,NaN,0,N,N,NaN,N,P\nP,Raba Automotive Holding Plc,3711,Engel General Developers Ltd,6552,None,None,None,None,None,None,None,None,None,None,None,None,None,None,new_entrant,-1975,1975,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,2
2002-01-08,88691.0,Manufacturing Services,MNF,AS,Transportation Equipment,Completed/Signed,Hyundai Corp\nMitsubishi Corp\nDaimlerChrysler AG,"Hyundai Corp, located in\nSeoul, South Korea, ...","Whl automobiles,metal prods\nMnfr,whl auto,che...",3711,3711,NaN,5051\n5012\n5084\n5191\n5088\n5065\n5169\n6719...,5051\n2899\n3711,3.0,No,NaN,No,267\n60520\n382724,Asian Wall Street Journal\nDuns Asia/Pacific-K...,0,N,N,NaN,N,P\nP\nP,Hyundai Corp,5051,Mitsubishi Corp,2899,DaimlerChrysler AG,3711,None,None,None,None,None,None,None,None,None,None,None,None,new_entrant,-1971,1971,0,267.0,60520.0,382724.0,NaN,NaN,NaN,NaN,NaN,NaN,True,True,11.903866,3
2002-01-08,88699.0,Manufacturing Services,MNF,AS,Transportation Equipment,Pending,Proton\nGold Star Heavy Ind Mnfr,Manufacture and retail motor\nvehicles; wholes...,"Mnfr,ret motor vehicles\nMnfr auto parts",3714,3714,NaN,3711\n5511\n5013\n3714,3711\n3714,2.0,No,NaN,No,\n,NaN,0,N,N,NaN,N,P\nV,Proton,3711,Gold Star Heavy Ind Mnfr,3714,None,None,None,None,None,None,None,None,None,None,None,None,None,None,new_entrant,-1971,1971,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-02,135098.0,Automotive Services,AUT,AS,Transportation Equipment,Completed/Signed,GAZ\nMersa Otomotiv Motorlu Araclar,"GAZ OAO, located in Nizhnii\nNovgorod, Russian...","Manufacture,whl motor vehicles\nMnfr,wholesale...",3711,3711\n5012,NaN,3711\n5012\n3711\n5012,3711\n3711,2.0,No,NaN,No,\n,NaN,0,N,N,TT,Y,P\nV,GAZ,3711,Mersa